# Image preference

- **Goal**: Show a standard workflow for working with complex multi-modal preference datasets like image question answering.
- **Dataset**: [RLAIF-V](https://huggingface.co/datasets/openbmb/RLAIF-V-Dataset), large-scale multimodal feedback dataset. The dataset provides high-quality feedback with a total number of 83,132 preference pairs for image question answering.
- **Libraries**: [datasets](https://github.com/huggingface/datasets), [sentence-transformers](https://github.com/UKPLab/sentence-transformers)
- **Components**: [ImageField](https://docs.argilla.io/latest/reference/argilla/settings/fields/#src.argilla.settings._field.ImageField), [TextQuestion](https://docs.argilla.io/latest/reference/argilla/settings/questions/#src.argilla.settings._question.TextQuestion), [VectorField](https://docs.argilla.io/dev/reference/argilla/settings/vectors/#rgvectorfield), [TermsMetadataProperty](https://docs.argilla.io/dev/reference/argilla/settings/metadata_property/?h=#rgtermsmetadataproperty)

## Getting started

### Deploy the Argilla server

If you already have deployed Argilla, you can skip this step. Otherwise, you can quickly deploy Argilla following [this guide](../getting_started/quickstart.md).

### Set up the environment

To complete this tutorial, you need to install the Argilla SDK and a few third-party libraries via `pip`.

In [ ]:
!pip install argilla

In [ ]:
!pip install "sentence-transformers>3,<4"

Let's make the required imports:

In [24]:
import argilla as rg

import io
import base64
import json
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, Dataset

You also need to connect to the Argilla server using the `api_url` and `api_key`.

In [3]:
# Replace api_url with your url if using Docker
# Replace api_key if you configured a custom API key
# Uncomment the last line and set your HF_TOKEN if your space is private
client = rg.Argilla(
    api_url="https://[your-owner-name]-[your_space_name].hf.space",
    api_key="owner.apikey",
    # headers={"Authorization": f"Bearer {HF_TOKEN}"}
)

## Vibe check the dataset

We will have a look at [the dataset](https://huggingface.co/datasets/openbmb/RLAIF-V-Dataset) to understand its structure and the kind of data it contains. We do this by using [the embedded Hugging Face Dataset Viewer](https://huggingface.co/docs/hub/main/en/datasets-viewer-embed).

<iframe
  src="https://huggingface.co/datasets/openbmb/RLAIF-V-Dataset/embed/viewer/default/train"
  frameborder="0"
  width="100%"
  height="560px"
></iframe>

## Configure and create the Argilla dataset

Now, we will need to configure the dataset. In the settings, we can specify the guidelines, fields, and questions. Because of the complexity of the task we will also add metadata and vectors. We will add one vector to represent the semantic meaning of the `question`. We will be adding two metadata fields that store the `original_dataset` and the type of task the question belongs to which can be obtained from `origin_split`. 

!!! note
    Check this [how-to guide](../how_to_guides/dataset.md) to know more about configuring and creating a dataset.

In [5]:
settings = rg.Settings(
    guidelines="The goal is to assess if the answers are correct and update them where needed.",
    fields=[
        rg.ImageField(
            name="image",
            title="An image of a certain object, state or action.",
        ),
        rg.TextField(
            name="question",
            title="A question about the image, intended to be answered.",
        )
    ],
    questions=[
        rg.TextQuestion(
            name="chosen",
            title="The chosen answer to the question.",
        ),
        rg.TextQuestion(
            name="rejected",
            title="The rejected answer to the question.",
        )
    ],
    metadata=[
        rg.TermsMetadataProperty(name="origin_dataset", title="Origin dataset"),
        rg.TermsMetadataProperty(name="task_type", title="Task type"),
    ],
    vectors=[
        rg.VectorField(name="question_vector", dimensions=384),
    ]
)

Let's create the dataset with the name and the defined settings:

In [ ]:
dataset = rg.Dataset(
    name="image_preference_dataset",
    settings=settings,
)
dataset.create()

## Add records

Even if we have created the dataset, it still lacks the information to be annotated (you can check it in the UI). We will use the `openbmb/RLAIF-V-Dataset` dataset from [the Hugging Face Hub](https://huggingface.co/datasets/openbmb/RLAIF-V-Dataset). Specifically, we will use `100` examples. Because we are dealing with a potentially large image dataset, we will set `streaming=True` to avoid loading the entire dataset into memory and iterate over the data to lazily load it.

!!! tip
    When working with Hugging Face dataset you can set `Image(decode=False)` so that we can get [public image URLs](https://huggingface.co/docs/datasets/en/image_load#local-files), however, this depends on the dataset.

In [ ]:
n_rows = 100
hf_dataset = load_dataset("openbmb/RLAIF-V-Dataset", streaming=True)
dataset_rows = []
count = 0
for row in hf_dataset["train"]:
    dataset_rows.append(row)
    count += 1
    if count >= n_rows:
        break
dataset_rows
hf_dataset = Dataset.from_list(dataset_rows)
hf_dataset

Let's have a look at the first image in the dataset.

In [19]:
hf_dataset[0]

{'ds_name': 'RLAIF-V',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 'question': 'Who is more likely to use these tools a leather crafter or a paper crafter?',
 'chosen': 'A leather crafter is more likely to use these tools. The image shows various crafting tools, including scissors and a hole punch, which are commonly used in leatherworking projects. Leather is a material that requires cutting, shaping, and precise hole-punching techniques to create desired designs or patterns. In contrast, paper crafters typically use different types of tools, such as adhesives, decorative papers, or specialized cutting machines like the Silhouette Cameo, for their projects.',
 'rejected': 'A leather crafter is more likely to use these tools as they consist of a hole punch, scissors, and a knife. These items are typically used in crafting projects involving fabric or leather materials for various designs and patterns. Paper crafters may also benefit from some of these to

### Convert PIL image to base64

As we can see, the image is a PIL Image. In order to use it in in Argilla, we need to convert them to a base64 string.

In [ ]:
def pil_to_data_uri(batch):
    data_uri = []
    for image in batch["image"]:
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        data_uri.append(f"data:image/png;base64,{img_str}")
    batch["image_data_uri"] = data_uri
    return batch

hf_dataset_with_base64= hf_dataset.map(pil_to_data_uri, batched=True)
hf_dataset_with_base64[0]

### Retrieve values nested in JSON

The question type values are nested in a JSON object. We can obtain them by looping through the data and getting the `origin_split` column.

In [ ]:
def retrieve_type_from_json(batch):
    loaded_json = [json.loads(x) for x in batch["origin_split"]]
    batch["task_type"] = [x["type"] for x in loaded_json]
    return batch

hf_dataset_with_base64_task= hf_dataset_with_base64.map(retrieve_type_from_json, batched=True)
hf_dataset_with_base64_task[0]

### Create vectors

We will use the `sentence-transformers` library to create vectors for the questions. We will use the `TaylorAI/bge-micro-v2` model which strikes a good balance between speed and performance. Note that we also need to convert the vectort to a `list` to store it in the Argilla dataset.

In [ ]:
model = SentenceTransformer("TaylorAI/bge-micro-v2")

def encode_questions(batch):
    vectors_as_numpy = model.encode(batch["question"])
    batch["question_vector"] = [x.tolist() for x in vectors_as_numpy]
    return batch

hf_dataset_with_base64_task_vectors = hf_dataset_with_base64_task.map(encode_questions, batched=True)
hf_dataset_with_base64_task_vectors[0]

### Log to Argilla


We will easily add them to the dataset using `log` and the mapping, where we indicate that the column `image_data_uri` is the data that should be added to the field `image`. We are also adding an `id` column to the record, so we can easily backtrack the record to the external data source.

In [ ]:
hf_dataset = hf_dataset.add_column("id", range(len(hf_dataset)))
dataset.records.log(records=hf_dataset, mapping={
    "image_data_uri": "image",
    "idx": "id",
    "question": "question",
    "chosen": "chosen",
    "rejected": "rejected",
    "task_type": "task_type",
    "question_vector": "question_vector",
    "origin_dataset": "origin_dataset"
})

Voilà! We have also added the suggestions to the dataset for the `chosen` `rejected` pairs, and they will appear in the UI marked with a ✨. 

## Evaluate with Argilla

Now, we can start the annotation process. Just open the dataset in the Argilla UI and start annotating the records. If the suggestions are correct, you can just click on `Submit`. Otherwise, you can select the correct label.

!!! note
    Check this [how-to guide](../how_to_guides/annotate.md) to know more about annotating in the UI.

## Conclusions

In this tutorial, we present an end-to-end example of a image classification task. This serves as the base, but it can be performed iteratively and seamlessly integrated into your workflow to ensure high-quality curation of your data and improved results.

We started by configuring the dataset, adding records, as an example, to add suggestions. After the annotation process, we trained a new model with the annotated data and updated the remaining records with the new suggestions.